# COMP47590: Advanced Machine Learning
# Assignment 1: Implementing Perceptrons

- Student 1 Name: Carl Fabian Winkler
- Student 1 Number: 20207528
- Student 2 Name: David Moreno Boras
- Student 2 Number: 0815

## Import Packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.base import BaseEstimator, ClassifierMixin

"""
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.utils.validation import check_X_y, check_array, check_is_fitted, check_random_state
from sklearn.utils.multiclass import unique_labels
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample"""

'\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.model_selection import train_test_split\n\nfrom sklearn.utils.validation import check_X_y, check_array, check_is_fitted, check_random_state\nfrom sklearn.utils.multiclass import unique_labels\nfrom sklearn import preprocessing\nfrom sklearn import metrics\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.utils import resample'

In [10]:
np.square(3)

9

## Task 1: The Perceptron Classifier

Define the PerceptronClassifier class

In [ ]:
class Layer():
    def init(activation = 'Sigmoid'):
        self.activation = activation
        
    def forward(x):
        z = inputs.dot(weight) + bias
        
        if activation = 'Sigmoid':
            out = 1 / (1 + np.exp(-z))
        else if activation = 'ReLu':
            out = np.maximum(z, 0)
        else if activation = 'TanH':
            out = np.tanh(z)
        else:
            out = z
        
        self.cache = (x, W, b, z)
        
        return out    
    
    def backward(self, d_out):
        inputs, weight, bias, z = self.cache
        
        if activation = 'Sigmoid':
            d_act = d_out * (1 / (1 + np.exp(-z))) * (1 - 1 / (1 + np.exp(-z)))
        else if activation = 'ReLu':
            d_act = d_out * (z > 0)
            
        else if activation = 'TanH':
            d_act = d_out * np.square(z)
        else:
            d_act = z
            
        d_inputs = d_act @ weight.T
        d_weight = inputs.T @ d_out
        d_bias = d_out.sum(axis=0)
        
        return d_inputs, d_weight, d_bias


class PerceptronClassifier(BaseEstimator, ClassifierMixin):
    
    """
    Parameters
    ----------
    Attributes
    ----------
    Notes
    -----
    See also
    --------
    Examples
    --------
    """
    # Constructor for the classifier object
    def __init__(self):

        """Setup a Perceptron classifier .
        Parameters
        ----------
        Returns
        -------
        """     

        # Initialise ranomd state if set
        self.random_state = random_state
        
        # Initialise class variabels
        
    
    # The fit function to train a classifier
    def fit(self, X, y):
        # WRITE CODE HERE
        

    # The predict function to make a set of predictions for a set of query instances
    def predict(self, X):
        # WRITE CODE HERE
    
    # The predict_proba function to make a set of predictions for a set of query instances. This returns a set of class distributions.
    def predict_proba(self, X):
        # WRITE CODE HERE

## Task 2: Evaluation

Load the Diabethic Retinopathy dataset

Evaluate the perfomrance of the perceptron classifier on the daibetic retinopathy dataset.

## Task 3 & 4: Add Different Activations & Regularisation

Reimplement the PerceptronClassifier class adding an activation function option and L2 regularisation. 

In [ ]:
class PerceptronClassifier2(BaseEstimator, ClassifierMixin):
    """
    """
    # Constructor for the classifier object
    def __init__(self):

        """Setup a Perceptron classifier .
        Parameters
        ----------

        
        Returns
        -------

        """     

        # Initialise ranomd state if set
        self.random_state = random_state
        
        # Initialise class variabels

        
    # The fit function to train a classifier
    def fit(self, X, y):
        
        # WRITE CODE HERE
        

    # The predict function to make a set of predictions for a set of query instances
    def predict(self, X):

        # WRITE CODE HERE
    
    # The predict_proba function to make a set of predictions for a set of query instances. This returns a set of class distributions.
    def predict_proba(self, X):
        
        # WRITE CODE HERE

Load the dataset and explore it.

## Task 5: Reflect on the Performance of the Different Models Evaluated

Perform hyper-parameter tuning and evaluate models. 